# Evolution of urban patterns: urban morphology as an open reproducible data science

## Urban morphometrics

This is the first notebook in a sequence of three. The notebook downloads data from OpenStreetMap, generates morphological tessellation and compute a set of morphometric characters.

It requires `data/case_studies.csv` input with origins of case studies.

Date: February 5, 2021

---

We start with the import of libraries used in this notebook.

In [1]:
import pandas as pd
import geopandas as gpd
import momepy as mm
import pyproj
import osmnx
import libpysal
from shapely.geometry import box
from time import time

We will use `osmnx` to download data and specify the timestamp `2021-02-05` to ensure the script always downloads the same data from overpass.

In [2]:
osmnx.config(overpass_settings='[out:json][timeout:90][date:"2021-02-05T00:00:00Z"]')

The only input of the whole analysis is the CSV with case studies.

In [3]:
cases = pd.read_csv("data/case_studiesZW.csv")
cases

,case,period,origin
0,City Centre,Eastern Zone,"(86.14787941191288, 23.667200105645797)"
1,Cuffe Parade,West Zone,"(72.82000253375675, 18.91442395337802)"
2,Ballard Estate,West Zone,"(72.83688268053827, 18.936504427534746)"
3,Nariman Point,West Zone,"(72.82480722132986, 18.925214568881227)"
4,Bandra Kurla Complex,West Zone,"(72.8627395804263, 19.066314041680574)"
5,CBD Belapur,West Zone,"(73.03629797764636, 19.02250701461816)"
6,Nagar Road,West Zone,"(73.93671275054801, 18.552918615853407)"
7,Viman nagar,West Zone,"(73.91370983261332, 18.569540936145668)"
8,Deccan Gymkhana,West Zone,"(73.8364547266035, 18.516500373211034)"
9,Yerwada Nagar,West Zone,"(73.88371376101495, 18.55895076622647)"


The following cell loops over all cases in `cases` DataFrame and does the sequence of the following steps:

1. Download building footprints within 800 m radius around the origin of the case study.
2. Reproject and preprocess them in a way which results in a GeoDataFrame of Polygon geometries.
3. Download street network within 1000 m radius around the origin of the case study.
4. Reproject and convert street network from graph to GeoDataFrame
5. Generate [morphological tessellation](http://docs.momepy.org/en/stable/user_guide/elements/tessellation.html) based on building footprints.
6. Measure a selection of morphometric characters either using GeoPandas or momepy.
7. Link all characters to morphological tessellation cells.
8. Save the result to GeoPackage.

In [4]:
%%capture --no-stdout

for i, coords in cases.origin.iteritems():
    s = time()
    """
    donwload and process elements of urban form
    """
    # download buildings from OSM
    point = tuple(map(float, coords[1:-1].split(', ')))[::-1]
    buildings = osmnx.geometries.geometries_from_point(point, dist=800, tags={'building':True})
    buildings = osmnx.projection.project_gdf(buildings)
    buildings = buildings[buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]
    buildings = buildings.reset_index(drop=True).explode().reset_index(drop=True)
    buildings = buildings[["geometry"]]

    # download streets from OSM
    streets_graph = osmnx.graph_from_point(point, 1000, network_type='drive')
    streets_graph = osmnx.get_undirected(streets_graph)
    streets_graph = osmnx.projection.project_graph(streets_graph)
    streets = osmnx.graph_to_gdfs(streets_graph, nodes=False, edges=True,
                                            node_geometry=False, fill_edge_geometry=True)
    streets = streets[["geometry"]]

    # check CRS
    assert buildings.crs.equals(streets.crs)

    # generate tessellation
    buildings['uID'] = range(len(buildings))
    tessellation = mm.Tessellation(buildings, "uID", limit=box(*buildings.total_bounds), verbose=False).tessellation

    """
    measure morphometric characters
    """
    # cell area
    tessellation['cell_area'] = tessellation.area

    # building area
    buildings['blg_area'] = buildings.area

    # coverage area ratio
    tessellation["car"] = mm.AreaRatio(tessellation, buildings, "cell_area", buildings.area, "uID").series

    # segment length
    streets["length"] = streets.length

    # segment linearity
    streets["linearity"] = mm.Linearity(streets, verbose=False).series

    # street profile
    profile = mm.StreetProfile(streets, buildings, distance=3)
    streets["width"] = profile.w
    streets["width_deviation"] = profile.wd
    streets["openness"] = profile.o

    # perimeter wall
    buildings["wall"] = mm.PerimeterWall(buildings, verbose=False).series

    # generate binary contiguity-based W objects of first order and inclusive order 3
    W1 = libpysal.weights.Queen.from_dataframe(tessellation, ids="uID")
    W3 = mm.sw_high(k=3, weights=W1)

    # building adjacency
    buildings["adjacency"] = mm.BuildingAdjacency(buildings, W3, "uID", verbose=False).series

    # interbuilding distance
    buildings['neighbour_distance'] = mm.NeighborDistance(buildings, W1, 'uID', verbose=False).series

    # generate networkx.MultiGraph object
    G = mm.gdf_to_nx(streets)

    # meshedness
    G = mm.meshedness(G, verbose=False)

    # convert networkx.MultiGraph to geopandas.GeoDataFrames
    nodes, edges = mm.nx_to_gdf(G)

    """
    link all characters to tessellation cells
    """
    # merge street network edges based on proximity
    edges["nID"] = range(len(edges))
    buildings["nID"] = mm.get_network_id(buildings, edges, "nID", 500, verbose=False)
    buildings = buildings.merge(edges.drop(columns="geometry"), on="nID", how="left")

    # merge nodes based on edge ID and proximity
    buildings["nodeID"] = mm.get_node_id(buildings, nodes, edges, "nodeID", "nID", verbose=False)
    buildings = buildings.merge(nodes.drop(columns="geometry"), on="nodeID", how="left")

    # merge buildings based on a shared attribute
    tessellation = tessellation.merge(buildings.drop(columns="geometry"), on="uID", how="left")

    """
    save all to GeoPackage
    """
    tessellation.to_file(f"data/{cases.loc[i, 'case']}.gpkg", layer="tessellation", driver="GPKG")
    buildings.to_file(f"data/{cases.loc[i, 'case']}.gpkg", layer="buildings", driver="GPKG")
    edges.to_file(f"data/{cases.loc[i, 'case']}.gpkg", layer="edges", driver="GPKG")
    nodes.to_file(f"data/{cases.loc[i, 'case']}.gpkg", layer="nodes", driver="GPKG")

    print(f"{cases.loc[i, 'case']} done in {time() - s} seconds.")

City Centre done in 4.349499225616455 seconds.
Cuffe Parade done in 17.537861108779907 seconds.
Ballard Estate done in 32.662063121795654 seconds.
Nariman Point done in 25.86450171470642 seconds.
Bandra Kurla Complex done in 16.64039182662964 seconds.
CBD Belapur done in 33.45376253128052 seconds.
Nagar Road done in 115.71827030181885 seconds.
Viman nagar done in 31.231408834457397 seconds.
Deccan Gymkhana done in 59.2099449634552 seconds.
Yerwada Nagar done in 20.427721977233887 seconds.
Baner gaon done in 27.76312255859375 seconds.
Bardi done in 9.584759712219238 seconds.
Gujarat International Finance Tec-City done in 1.72226881980896 seconds.
Sector 17 done in 23.49785351753235 seconds.
Connaught Place done in 30.160667419433594 seconds.
Nehru Place done in 63.80818319320679 seconds.
T Nagar done in 117.91412949562073 seconds.
Anna Salai done in 88.20573115348816 seconds.
Parry's Corner, George Town done in 63.84361743927002 seconds.
Nungambakkam done in 62.41083860397339 seconds.
H

In [5]:
# Alert
i=0
while i < 5:
    # Play sound when done
    from playsound import playsound
    playsound("C:/Users/HP/Downloads/beep.mp3")
    i=i+2


    Error 259 for command:
        play C:/Users/HP/Downloads/beep.mp3 wait
    The driver cannot recognize the specified command parameter.

    Error 263 for command:
        close C:/Users/HP/Downloads/beep.mp3
    The specified device is not open or is not recognized by MCI.
Failed to close the file: C:/Users/HP/Downloads/beep.mp3


PlaysoundException: 
    Error 259 for command:
        play C:/Users/HP/Downloads/beep.mp3 wait
    The driver cannot recognize the specified command parameter.